In [8]:
library(tidyverse) 
library(dplyr)
library(ggplot2)
library(ggpubr)

print("Working directory before:")
getwd()

setwd("/home/strawberry/Documents/Collaborations/sinusite_edwin/data") # changing work directory to data 
print("Working directory after:")
getwd()

# Loading reference file (linking refseq accession number and taxID) - this file was obtained parsing gff3 file provided by NCBI 
# RefseqTaxID <- read.delim("./RefseqTaxID.txt", h=T)
# head(RefseqTaxID)
SILVATaxID <- read.csv("./silva_taxonomy.csv", h=T)
SILVATaxID$id <- gsub("\\..*", "", SILVATaxID$id) # Clean the 'id' column (remove everything after the first dot)
head(SILVATaxID)


# Loading TaxID full lineage file - no need for SILVA db, already contains id linked to full taxonomy in the above df 
# TaxID_FullLineage <- read.delim("./referencetable_taxonomy_RefseqNCBI_16S.txt", quote = "")
# head(TaxID_FullLineage)

# Loading metadata file including read counts after quality check
metadata_full_clean <- read.delim("./metadata_full_clean.tsv", h=T) 
head(metadata_full_clean)

[1] "Working directory before:"


[1] "/home/strawberry/Documents/Collaborations/sinusite_edwin/data"

[1] "Working directory after:"


[1] "/home/strawberry/Documents/Collaborations/sinusite_edwin/data"

,id,Kingdom,Phylum,Class,Order,Family,Genus,Species
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,AB000393,Bacteria,Pseudomonadota,Gammaproteobacteria,Enterobacterales,Vibrionaceae,Vibrio,Vibrio halioticoli
2,AB000480,Bacteria,Pseudomonadota,Alphaproteobacteria,Rhodospirillales,Rhodospirillaceae,Insolitispirillum,Insolitispirillum peregrinum
3,AB001783,Bacteria,Chlamydiota,Chlamydiia,Chlamydiales,Chlamydiaceae,Chlamydia,Chlamydia abortus
4,FZ423313,Bacteria,Bacillota,Bacilli,Bacillales,Bacillaceae,Anoxybacillus,unidentified
5,HG529990,Bacteria,Bacteroidota,Bacteroidia,Cytophagales,Cyclobacteriaceae,Algoriphagus,Algoriphagus sp. AK58
6,AB001778,Bacteria,Chlamydiota,Chlamydiia,Chlamydiales,Chlamydiaceae,Chlamydia,Chlamydia psittaci


,ID_Sample,SNOT22,SNOT22nasal,LK_Total,LK_Secret,ClinicCtrl,AntUse,Cortuse,Patient,Filename,⋯,Timepoint,Sex,Age,Atopic_Symptoms,Asthma,NSAID_Intolerance,Total_IgE,Blood_Eosinophilia,Tissue_Eosinophilia,Final_LundMackay
,<chr>,<int>,<int>,<int>,<int>,<chr>,<int>,<int>,<chr>,<chr>,⋯,<chr>,<int>,<dbl>,<int>,<int>,<int>,<dbl>,<int>,<int>,<int>
1,0001967K_T1,38,12,4,2,NC,1,1,0001967K,0001967K_T1.fastq,⋯,T1,1,58.7,1,1,0,38,100,10,15
2,0001967K_T3,96,26,10,4,NC,1,0,0001967K,0001967K_T3.fastq,⋯,T3,1,58.7,1,1,0,38,100,10,15
3,0001967K_T6,79,19,6,4,NC,1,1,0001967K,0001967K_T6.fastq,⋯,T6,1,58.7,1,1,0,38,100,10,15
4,0088303G_T0,102,37,8,2,NC,NA,0,0088303G,0088303G_T0.fastq,⋯,T0,2,60.3,1,1,0,667,800,58,22
5,0088303G_T1,32,7,4,1,NC,1,0,0088303G,0088303G_T1.fastq,⋯,T1,2,60.3,1,1,0,667,800,58,22
6,0088303G_T3,NA,NA,NA,NA,NA,0,0,0088303G,0088303G_T3.fastq,⋯,T3,2,60.3,1,1,0,667,800,58,22


In [9]:
# Make a repository with kma output
fragfiles <- list.files("./fragfiles", pattern = ".frag$")
fragfiles

[1] "0001967K_T1_filt_kma.frag"   "0001967K_T3_filt_kma.frag"  
  [3] "0001967K_T6_filt_kma.frag"   "0018303G_T0_filt_kma.frag"  
  [5] "0088303G_T0_filt_kma.frag"   "0088303G_T1_filt_kma.frag"  
  [7] "0088303G_T3_filt_kma.frag"   "0110059G_T0_filt_kma.frag"  
  [9] "0110059G_T1_filt_kma.frag"   "0110059G_T12_filt_kma.frag" 
 [11] "0110059G_T3_filt_kma.frag"   "0110059G_T6_filt_kma.frag"  
 [13] "0120695B_T0_filt_kma.frag"   "0130765A_T0_filt_kma.frag"  
 [15] "0130765A_T1_filt_kma.frag"   "0130765A_T12_filt_kma.frag" 
 [17] "0130765A_T3_filt_kma.frag"   "0130765A_T6_filt_kma.frag"  
 [19] "0207720E_T0_filt_kma.frag"   "0207720E_T1_filt_kma.frag"  
 [21] "0207720E_T12_filt_kma.frag"  "0207720E_T3_filt_kma.frag"  
 [23] "0236549G_T0_filt_kma.frag"   "0236549G_T12_filt_kma.frag" 
 [25] "0236549G_T3_filt_kma.frag"   "0236549G_T6_filt_kma.frag"  
 [27] "0313863D_T0_filt_kma.frag"   "0313863D_T1_filt_kma.frag"  
 [29] "0313863D_T12_filt_kma.frag"  "0313863D_T3_filt_kma.frag"  
 [31] "0332596H_PreC_filt_kma.frag" "0332596H_T1_filt_kma.frag"  
 [33] "0332596H_T6_filt_kma.frag"   "0358613K_T0_filt_kma.frag"  
 [35] "0358613K_T1_filt_kma.frag"   "0358613K_T3_filt_kma.frag"  
 [37] "0358613K_T6_filt_kma.frag"   "0492882D_T0_filt_kma.frag"  
 [39] "0492882D_T1_filt_kma.frag"   "0492882D_T12_filt_kma.frag" 
 [41] "0492882D_T3_filt_kma.frag"   "0492882D_T6_filt_kma.frag"  
 [43] "0514164A_T0_filt_kma.frag"   "0514164A_T1_filt_kma.frag"  
 [45] "0514164A_T3_filt_kma.frag"   "0514164A_T6_filt_kma.frag"  
 [47] "0518952I_T0_filt_kma.frag"   "0561785J_T0_filt_kma.frag"  
 [49] "0584352K_T0_filt_kma.frag"   "0584352K_T1_filt_kma.frag"  
 [51] "0584352K_T3_filt_kma.frag"   "0584352K_T6_filt_kma.frag"  
 [53] "0689227I_T0_filt_kma.frag"   "0689227I_T1_filt_kma.frag"  
 [55] "0689227I_T12_filt_kma.frag"  "0689227I_T3_filt_kma.frag"  
 [57] "0689227I_T6_filt_kma.frag"   "0732163H_T0_filt_kma.frag"  
 [59] "0732163H_T3_filt_kma.frag"   "0732163H_T6_filt_kma.frag"  
 [61] "0780068I_T0_filt_kma.frag"   "0780068I_T12_filt_kma.frag" 
 [63] "0780068I_T3_filt_kma.frag"   "0780068I_T6_filt_kma.frag"  
 [65] "0784533E_T0_filt_kma.frag"   "0784533E_T1_filt_kma.frag"  
 [67] "0784533E_T3_filt_kma.frag"   "0784533E_T6_filt_kma.frag"  
 [69] "0878122D_T0_filt_kma.frag"   "0878122D_T3_filt_kma.frag"  
 [71] "0878122D_T6_filt_kma.frag"   "0928507A_T0_filt_kma.frag"  
 [73] "0974256K_T0_filt_kma.frag"   "0974256K_T1_filt_kma.frag"  
 [75] "0980601G_T0_filt_kma.frag"   "0980601G_T1_filt_kma.frag"  
 [77] "0980601G_T12_filt_kma.frag"  "0980601G_T3_filt_kma.frag"  
 [79] "0980601G_T6_filt_kma.frag"   "0990961D_T0_filt_kma.frag"  
 [81] "0990961D_T1_filt_kma.frag"   "0990961D_T12_filt_kma.frag" 
 [83] "0990961D_T3_filt_kma.frag"   "0990961D_T6_filt_kma.frag"  
 [85] "1190552D_T0_filt_kma.frag"   "1190552D_T12_filt_kma.frag" 
 [87] "1190552D_T3_filt_kma.frag"   "1272679D_T0_filt_kma.frag"  
 [89] "1272679D_T1_filt_kma.frag"   "1272679D_T12_filt_kma.frag" 
 [91] "1272679D_T3_filt_kma.frag"   "1272679D_T6_filt_kma.frag"  
 [93] "1377897D_T0_filt_kma.frag"   "1377897D_T1_filt_kma.frag"  
 [95] "1377897D_T12_filt_kma.frag"  "1377897D_T3_filt_kma.frag"  
 [97] "1427495A_T0_filt_kma.frag"   "1427495A_T12_filt_kma.frag" 
 [99] "1427495A_T3_filt_kma.frag"   "1427495A_T6_filt_kma.frag"  
[101] "1512536E_T0_filt_kma.frag"   "1512536E_T1_filt_kma.frag"  
[103] "1512536E_T12_filt_kma.frag"  "1512536E_T3_filt_kma.frag"  
[105] "1512536E_T6_filt_kma.frag"   "1526680J_T1_filt_kma.frag"  
[107] "1526680J_T12_filt_kma.frag"  "1526680J_T3_filt_kma.frag"  
[109] "1526680J_T6_filt_kma.frag"   "1532317F_T0_filt_kma.frag"  
[111] "1532317F_T12_filt_kma.frag"  "1532317F_T3_filt_kma.frag"  
[113] "1532317F_T6_filt_kma.frag"   "1552332D_T0_filt_kma.frag"  
[115] "1552332D_T1_filt_kma.frag"   "1552332D_T12_filt_kma.frag" 
[117] "1552332D_T3_filt_kma.frag"   "1553248E_T0_filt_kma.frag"  
[119] "1553248E_T1_filt_kma.frag"   "1553248E_T3_filt_kma.frag"  
[121] "1553248E_T6_filt_kma.frag"   "15576

In [10]:
dfall <- NULL  # Start with an empty dataframe
tab <- read.delim("./fragfiles/1628845C_T1_filt_kma.frag", h=F)
print(head(tab))

In [11]:
freqtab <- table(gsub("\\..*", "", tab$V6))
print(freqtab)


    AB009937     AB233328     AB680534     AB680900     AB681292     AB681715 
           4            1            1            2            2            4 
    AE015929 AFGU01000017 AGEF01000001 AGEF01000005 AGEF01000016 AIYD01000002 
          19           17           12          122           18            1 
    AJ308297     AJ938182 AJCB01000004     AM600682     AM697499     AP017895 
           1            1            1            1            1            1 
    AP018338     AP018922     AY281086     AY308046     AY560519     AY587778 
           2            8            2            1            1            3 
    BX571857     CP000253     CP001837     CP002110     CP002727     CP003045 
           1            1            8            5            1            5 
    CP003979     CP005288     CP006937     CP009046     CP012011     CP012013 
           9            1            1           52            1            2 
    CP012972     CP012974     CP012978     CP014022

In [12]:
df <- data.frame(id = names(freqtab), Counts = as.numeric(freqtab))   
print(df)

              id Counts
1       AB009937      4
2       AB233328      1
3       AB680534      1
4       AB680900      2
5       AB681292      2
6       AB681715      4
7       AE015929     19
8   AFGU01000017     17
9   AGEF01000001     12
10  AGEF01000005    122
11  AGEF01000016     18
12  AIYD01000002      1
13      AJ308297      1
14      AJ938182      1
15  AJCB01000004      1
16      AM600682      1
17      AM697499      1
18      AP017895      1
19      AP018338      2
20      AP018922      8
21      AY281086      2
22      AY308046      1
23      AY560519      1
24      AY587778      3
25      BX571857      1
26      CP000253      1
27      CP001837      8
28      CP002110      5
29      CP002727      1
30      CP003045      5
31      CP003979      9
32      CP005288      1
33      CP006937      1
34      CP009046     52
35      CP012011      1
36      CP012013      2
37      CP012972      2
38      CP012974      3
39      CP012978      1
40      CP014022    100
41      CP014023

In [14]:
dfTaxID <- merge(SILVATaxID, df, by = "id")
print(head(dfTaxID))

        id  Kingdom    Phylum   Class            Order            Family
1 AB009937 Bacteria Bacillota Bacilli Staphylococcales Staphylococcaceae
2 AB233328 Bacteria Bacillota Bacilli Staphylococcales Staphylococcaceae
3 AB680534 Bacteria Bacillota Bacilli  Lactobacillales  Streptococcaceae
4 AB680900 Bacteria Bacillota Bacilli  Lactobacillales Carnobacteriaceae
5 AB681292 Bacteria Bacillota Bacilli Staphylococcales Staphylococcaceae
6 AB681715 Bacteria Bacillota Bacilli Staphylococcales Staphylococcaceae
           Genus                                   Species Counts
1 Staphylococcus Staphylococcus capitis subsp. urealyticus      4
2 Staphylococcus  Staphylococcus cohnii subsp. urealyticus      1
3  Streptococcus                  Streptococcus salivarius      1
4 Dolosigranulum                     Dolosigranulum pigrum      2
5 Staphylococcus                Staphylococcus epidermidis      2
6 Staphylococcus                     Staphylococcus aureus      4


In [15]:
# freqtable_tax <- merge(dfTaxID, TaxID_FullLineage, by.x = "TaxID", by.y = "id") 
# print(freqtable_tax)

In [18]:
dfTaxID$ID_Sample <- gsub("_filt_kma.frag", "", "1628845C_T1_filt_kma.frag") 
print(head(dfTaxID))

        id  Kingdom    Phylum   Class            Order            Family
1 AB009937 Bacteria Bacillota Bacilli Staphylococcales Staphylococcaceae
2 AB233328 Bacteria Bacillota Bacilli Staphylococcales Staphylococcaceae
3 AB680534 Bacteria Bacillota Bacilli  Lactobacillales  Streptococcaceae
4 AB680900 Bacteria Bacillota Bacilli  Lactobacillales Carnobacteriaceae
5 AB681292 Bacteria Bacillota Bacilli Staphylococcales Staphylococcaceae
6 AB681715 Bacteria Bacillota Bacilli Staphylococcales Staphylococcaceae
           Genus                                   Species Counts   ID_Sample
1 Staphylococcus Staphylococcus capitis subsp. urealyticus      4 1628845C_T1
2 Staphylococcus  Staphylococcus cohnii subsp. urealyticus      1 1628845C_T1
3  Streptococcus                  Streptococcus salivarius      1 1628845C_T1
4 Dolosigranulum                     Dolosigranulum pigrum      2 1628845C_T1
5 Staphylococcus                Staphylococcus epidermidis      2 1628845C_T1
6 Staphylococcus     

In [20]:
freqtable_tax_metadata <- merge(dfTaxID, metadata_full_clean, by = "ID_Sample")
print(head(freqtable_tax_metadata))

    ID_Sample       id  Kingdom    Phylum   Class            Order
1 1628845C_T1 AB009937 Bacteria Bacillota Bacilli Staphylococcales
2 1628845C_T1 AB233328 Bacteria Bacillota Bacilli Staphylococcales
3 1628845C_T1 AB680534 Bacteria Bacillota Bacilli  Lactobacillales
4 1628845C_T1 AB680900 Bacteria Bacillota Bacilli  Lactobacillales
5 1628845C_T1 AB681292 Bacteria Bacillota Bacilli Staphylococcales
6 1628845C_T1 AB681715 Bacteria Bacillota Bacilli Staphylococcales
             Family          Genus                                   Species
1 Staphylococcaceae Staphylococcus Staphylococcus capitis subsp. urealyticus
2 Staphylococcaceae Staphylococcus  Staphylococcus cohnii subsp. urealyticus
3  Streptococcaceae  Streptococcus                  Streptococcus salivarius
4 Carnobacteriaceae Dolosigranulum                     Dolosigranulum pigrum
5 Staphylococcaceae Staphylococcus                Staphylococcus epidermidis
6 Staphylococcaceae Staphylococcus                     Staphylococcus

In [25]:
freqtable_tax_metadata$RelativeAbundance <- ((freqtable_tax_metadata$Counts) / sum(freqtable_tax_metadata$Counts)) * 100
print(head(freqtable_tax_metadata))

print("Relative Abundance Summary:")
print(summary(freqtable_tax_metadata$RelativeAbundance))

    ID_Sample       id  Kingdom    Phylum   Class            Order
1 1628845C_T1 AB009937 Bacteria Bacillota Bacilli Staphylococcales
2 1628845C_T1 AB233328 Bacteria Bacillota Bacilli Staphylococcales
3 1628845C_T1 AB680534 Bacteria Bacillota Bacilli  Lactobacillales
4 1628845C_T1 AB680900 Bacteria Bacillota Bacilli  Lactobacillales
5 1628845C_T1 AB681292 Bacteria Bacillota Bacilli Staphylococcales
6 1628845C_T1 AB681715 Bacteria Bacillota Bacilli Staphylococcales
             Family          Genus                                   Species
1 Staphylococcaceae Staphylococcus Staphylococcus capitis subsp. urealyticus
2 Staphylococcaceae Staphylococcus  Staphylococcus cohnii subsp. urealyticus
3  Streptococcaceae  Streptococcus                  Streptococcus salivarius
4 Carnobacteriaceae Dolosigranulum                     Dolosigranulum pigrum
5 Staphylococcaceae Staphylococcus                Staphylococcus epidermidis
6 Staphylococcaceae Staphylococcus                     Staphylococcus

In [29]:
freqtable_tax_metadata <- freqtable_tax_metadata %>%
  group_by(ID_Sample) %>%
  mutate(AlignmentEffic = (sum(Counts) / unique(nreads)) * 100)

head(freqtable_tax_metadata)
print("Alignment Efficiency Summary:")
print(summary(freqtable_tax_metadata$AlignmentEffic))

ID_Sample,id,Kingdom,Phylum,Class,Order,Family,Genus,Species,Counts,⋯,Age,Atopic_Symptoms,Asthma,NSAID_Intolerance,Total_IgE,Blood_Eosinophilia,Tissue_Eosinophilia,Final_LundMackay,RelativeAbundance,AlignmentEffic
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,⋯,<dbl>,<int>,<int>,<int>,<dbl>,<int>,<int>,<int>,<dbl>,<dbl>
1628845C_T1,AB009937,Bacteria,Bacillota,Bacilli,Staphylococcales,Staphylococcaceae,Staphylococcus,Staphylococcus capitis subsp. urealyticus,4,⋯,37.8,1,0,0,58,200,160,17,0.05180676,119.0043
1628845C_T1,AB233328,Bacteria,Bacillota,Bacilli,Staphylococcales,Staphylococcaceae,Staphylococcus,Staphylococcus cohnii subsp. urealyticus,1,⋯,37.8,1,0,0,58,200,160,17,0.01295169,119.0043
1628845C_T1,AB680534,Bacteria,Bacillota,Bacilli,Lactobacillales,Streptococcaceae,Streptococcus,Streptococcus salivarius,1,⋯,37.8,1,0,0,58,200,160,17,0.01295169,119.0043
1628845C_T1,AB680900,Bacteria,Bacillota,Bacilli,Lactobacillales,Carnobacteriaceae,Dolosigranulum,Dolosigranulum pigrum,2,⋯,37.8,1,0,0,58,200,160,17,0.02590338,119.0043
1628845C_T1,AB681292,Bacteria,Bacillota,Bacilli,Staphylococcales,Staphylococcaceae,Staphylococcus,Staphylococcus epidermidis,2,⋯,37.8,1,0,0,58,200,160,17,0.02590338,119.0043
1628845C_T1,AB681715,Bacteria,Bacillota,Bacilli,Staphylococcales,Staphylococcaceae,Staphylococcus,Staphylococcus aureus,4,⋯,37.8,1,0,0,58,200,160,17,0.05180676,119.0043


[1] "Alignment Efficiency Summary:"
   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
    119     119     119     119     119     119 


In [31]:
write_tsv(freqtable_tax_metadata, "freqtable_tax_metadata.tsv")